# Load Files and libraries

In [17]:
import aicspylibczi
import napari
import numpy as np
import os
import pathlib
import tifffile
import time
from datetime import timezone, datetime, timedelta

import glob

from skimage import data, io
from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from scipy.ndimage import fourier_shift
from scipy.ndimage import shift

# Main script

In [23]:
%run main.py ./czi ./output --disable-subtract-background --y

Found the following image files in ./czi: 

./czi/align1.czi
./czi/align2.czi

--- Processing first CZI:  ['./czi/align1.czi']
info – czi dims_shape:  [{'X': (0, 614), 'Y': (0, 427), 'Z': (0, 1), 'C': (0, 5), 'T': (0, 1), 'M': (0, 4), 'S': (0, 1), 'B': (0, 1)}]
info – czi channels:  5
Mosaic 0 DONE
info – channel 0 read, and image processed
Mosaic 1 DONE
info – channel 1 read, and image processed
Mosaic 2 DONE
info – channel 2 read, and image processed
Mosaic 3 DONE
info – channel 3 read, and image processed
Mosaic 4 DONE
info – channel 4 read, and image processed
data shape  (5, 1825, 1995)
Shape of first image processed is:  (1, 5, 1825, 1995)
--- Processig CZI i: ['./czi/align2.czi']
info – czi dims_shape:  [{'X': (0, 214), 'Y': (0, 197), 'Z': (0, 1), 'C': (0, 5), 'T': (0, 1), 'M': (0, 6), 'S': (0, 1), 'B': (0, 1)}]
info – czi channels:  5
Mosaic 0 DONE
info – channel 0 read, and image processed
Mosaic 1 DONE
info – channel 1 read, and image processed
Mosaic 2 DONE
info – channel 2 

# CZI script

In [18]:
def get_files(source):
    return glob.glob(source + '/**/*.czi', recursive=True)

def list_files(source, files):
    file_names = '\n'.join(files)
    file_list = f'''Found the following image files in {source}: \n\n{file_names}\n'''

    print(file_list)

def ask_for_approval():
    hasApproval = False

    while not hasApproval:
        user_input = input('Continue with image processing for the above files? (Yes/No): ').strip().lower()

        if user_input == 'yes' or user_input == 'y':
            hasApproval = True
        elif user_input == 'no' or user_input == 'n':
            print('Terminating image processing.')
            exit()
        else:
            print('Please enter a valid option.')
            
def run(args):
    # gets the files and then gets the processed aligned images (through get_images) after shows through napari 
    # and finally saves a tif
    if args.time: run_time = time.monotonic()

    source = args.source
    files = get_files(source)

    list_files(source, files)

    if not args.yes:
        ask_for_approval()

    images = get_images(args, files)

    print('FINISHED, shape of images', np.shape(images))

    # show in napaari
    show(args, images)
    #save tif
    write(args, images)

    if args.time: print(timedelta(seconds=time.monotonic() - run_time))

# Image Processing script

In [19]:
def norm_by(x, min_, max_):
    norms = np.percentile(x, [min_, max_])
    i2 = np.clip((x - norms[0])/(norms[1]-norms[0]), 0, 1)
    return i2

def read(args, czi):
    #reads the czi images and does background subtraction or normalization on it. It returns a np.array of values.
    data = []
    dims_shape = czi.dims_shape()

    if not 'C' in dims_shape[0]:
        raise Exception("Image lacks Channels")

    channels = dims_shape[0]['C'][1]
    print('info – czi dims_shape: ', dims_shape)
    # M is mosaic: index of tile for compositing a scene
    print('info – czi channels: ', channels)

    read_time_total = 0
    subtract_background_time_total = 0

    for channel in range(channels):
        if args.time: read_time = time.monotonic()
        mosaic = czi.read_mosaic(C=channel, scale_factor=1)
        print('Mosaic', channel, 'DONE')
        if args.time: read_time_total += time.monotonic() - read_time
    # add option for not subtracting background,
    # normed_mosaic_data = mosaic[0, 0, :, :]

        if args.time: subtract_background_time = time.monotonic()
        if not args.disable_subtract_background:
            normed_mosaic_data = subtract_background(mosaic[0, 0, :, :])
            data.append(normed_mosaic_data)
        else:
            #print('info – use norm_by instead of subtract_background')
            normed_mosaic_data = norm_by(mosaic[0, 0, :, :], 5, 98) * 255
            data.append(normed_mosaic_data)
        if args.time: subtract_background_time_total += time.monotonic() - subtract_background_time

        print(f'''info – channel {channel} read, and image processed''') #subtraction or normalization

    if args.time:
        print('info – czi.read_mosaic time', timedelta(seconds=read_time_total))
        print('info – subtract_background time', timedelta(seconds=subtract_background_time_total))

    print('data shape ', np.shape(data))
    return data

def get_processed_czis(args, czis):
    #takes the np.array of czi and processes the array by doing background subtraction or normalization.
    processed_czis = []

    for czi in czis:
        processed_czis.append(read(args, czi))
    return processed_czis

def get_czis(files):
    # Takes the string of files and creates np.array of the czi images 
    # WARNING: the array consists of CZI images not transformed to values yet.
    czis = []

    for file in files:
        czis.append(aicspylibczi.CziFile(pathlib.Path(file)))

    # sorting?
    # "file handling: load files in order of numerical round, not alphab sorting"
    return np.array(czis)

def show(args, images):
    if not args.yes:
        with napari.gui_qt():
            viewer = napari.Viewer()
            # viewer = napari.view_image(images[0][0].astype(np.uint8))
            for czi in images:
                viewer.add_image(np.array(czi))

def write(args, imagesToShape):
    shape = np.shape(imagesToShape)
    newShape = shape[0] * shape[1]
    images = np.array(imagesToShape).reshape(1, newShape, shape[2], shape[3])
    images = np.array(imagesToShape).reshape(shape)
    #images = np.array(imagesToShape)
    #print(type(images))

    #print('np.array(images).flatten()', np.shape(images))
    if args.destination:
        if os.path.exists(args.destination):
            name = f'{datetime.now().strftime("%Y-%m-%d")}_{int(datetime.now(tz=timezone.utc).timestamp() * 1000)}'
            extension = 'ome.tif'
            file = f'{os.path.basename(args.destination)}/{name}.{extension}'

            with tifffile.TiffWriter(file) as tif:
            # additional metadata can be added, and in a more compatible format
            # axes is just an (incorrect) example
            # https://stackoverflow.com/questions/20529187/what-is-the-best-way-to-save-image-metadata-alongside-a-tif
            #for image in images:
            #tif.save(images, metadata={'axes':'TZCYX'}) <--- metadata does not work
                tif.save(images)
        else:
            print('destination path does not exist')

# Process script

In [20]:

def get_registration(args, processed_czis):
    if args.disable_registration:
        return processed_czis
    else:
        return align_images(args, processed_czis)


def get_images(args, files):
      # get the czi images with get_czis, then processes the images by background subtraction or normalization
      # Finally gets the aligned images with get_registration.

      #Process first image
    print('--- Processing first CZI: ', files[0].split())
    processed_czi0 = get_processed_czis(args, get_czis(files[0].split()))
    processed_czi0 = np.array(processed_czi0)
    print('Shape of first image processed is: ', np.shape(processed_czi0))

    #Register the first image as aligned image and use that as reference
    aligned_images = processed_czi0

    #process i images (i=1...R) and align them with the first image
    for file in files[1:]:
        print('--- Processig CZI i:', file.split())
        processed_czi = get_processed_czis(args,get_czis(file.split()))
        processed_czi = np.array(processed_czi)
        print('Shape of processed image i is: ', np.shape(processed_czi))

        #Align image0 and i together, returning an array of [2,C,X,Y]
        aligned_img = get_registration(args, np.concatenate((processed_czi0, processed_czi), axis = 0))
        #To remove the first image as it is already registered in aligned_images before the for
        aligned_img = np.delete(aligned_img, 0, axis = 0)
        aligned_images = np.concatenate((aligned_images, aligned_img), axis = 0)



    #OLD CODE delete all of the above and uncomment below for it to work
    #czis = get_czis(files)
    #processed_czis = get_processed_czis(args, czis)
    if args.time: aligned_images_time = time.monotonic()
    #aligned_images = get_registration(args, processed_czis)
    if args.time: print('info – image registration', timedelta(seconds=time.monotonic() - aligned_images_time))

    # print('aligned_images', np.shape(aligned_images))

    return aligned_images



# Registration script

In [21]:
## align images based on the first dapi provided
## 
## register_translation, is fast
def align_images(args, processed_czis):
    aligned_images = []

    for czi in processed_czis:
        #print('length of a', len(aligned_images))

        if len(aligned_images):
            dapi_target = aligned_images[0][-1]
            dapi_to_offset = czi[-1]
            print('dapi_target shape', np.shape(dapi_target))
            print('dapi_to_offset shape', np.shape(dapi_to_offset))
            shifted, error, diffphase = register_translation(dapi_target, dapi_to_offset, 100)

            print(f"Detected subpixel offset (y, x): {shifted}")

            alignedCzi = []

        for image_from_channel in czi:
            #print('channel size', np.shape(image_from_channel))

            corrected_image = shift(image_from_channel, shift=(shifted[0], shifted[1]), mode='constant')

            alignedCzi.append(corrected_image)

            print('transformed channels done')
            aligned_images.append(alignedCzi)

    else:
        aligned_images.append(czi)

    return aligned_images



# Subtract background

In [22]:
from skimage.morphology import white_tophat, black_tophat, disk

def subtract_background(image, radius=50, light_bg=False):
    str_el = disk(radius) #you can also use 'ball' here to get a slightly smoother result at the cost of increased computing time

    if light_bg:
        return black_tophat(image, str_el)
    else:
        return white_tophat(image, str_el)

#source: https://forum.image.sc/t/background-subtraction-in-scikit-image/39118

